[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/timz815/360-NLP-Project/blob/main/finetune.ipynb)

In [ ]:
# pip3 install unsloth
# pip3 install transformers datasets trl accelerate
# pip3 install torch torchvision torchaudio


In [1]:
# ==========
# load model
# ==========
from unsloth import FastLanguageModel

model_name = "unsloth/Qwen3-4B-unsloth-bnb-4bit"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=2048,
    load_in_4bit=True,
    device_map="auto",
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


C:\Users\timot\AppData\Local\Programs\Python\Python313\Lib\site-packages\triton\knobs.py:212: UserWarning: Failed to find cuobjdump.exe
  warnings.warn(f"Failed to find {binary}")
C:\Users\timot\AppData\Local\Programs\Python\Python313\Lib\site-packages\triton\knobs.py:212: UserWarning: Failed to find nvdisasm.exe
  warnings.warn(f"Failed to find {binary}")
C:\Users\timot\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
W1205 20:34:37.774000 28372 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
[tensorflow|WARNING]From C:\Users\timot\AppData\Local\Programs\Python\Python313\Lib\site-packages\tf_keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please 

🦥 Unsloth Zoo will now patch everything to make training faster!


C:\Users\timot\AppData\Local\Programs\Python\Python313\Lib\site-packages\unsloth_zoo\gradient_checkpointing.py:348: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:35.)
  GPU_BUFFERS = tuple([torch.empty(2*256*2048, dtype = dtype, device = f"{DEVICE_TYPE_TORCH}:{i}") for i in range(n_gpus)])


==((====))==  Unsloth 2025.11.4: Fast Qwen3 patching. Transformers: 4.57.1.
   \\   /|    NVIDIA GeForce RTX 4060 Ti. Num GPUs = 1. Max memory: 7.996 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [2]:
# ============
# load dataset
# ============
from datasets import Dataset
import json

jsonl_path = r"C:\Users\timot\Downloads\nlp training\movie_dialogue.jsonl"
data = [json.loads(line) for line in open(jsonl_path, encoding="utf-8")]

ds = Dataset.from_list(data)

In [3]:
# ===================
# split data 85/10/5
# ===================
# 10% test takes too long
splits = ds.train_test_split(test_size=0.05, seed=42)
test_ds = splits["test"]
train_val = splits["train"]

val_fraction = 0.10 / 0.95
val_splits = train_val.train_test_split(test_size=val_fraction, seed=42)
train_ds = val_splits["train"]
val_ds   = val_splits["test"]

print("Train:", len(train_ds))
print("Val:",   len(val_ds))
print("Test:",  len(test_ds))

raw_test = Dataset.from_list([
    {"english": ex["english"], "chinese": ex["chinese"]}
    for ex in test_ds
])



Train: 8023
Val: 945
Test: 473


In [4]:
# ==============
# format + token
# ==============
def format_example(ex, tokenizer):
    messages = [
        {"role": "user", "content": ex["chinese"]},
        {"role": "assistant", "content": ex["english"]},
        # for en to zh
        #{"role": "user", "content": ex["english"]},
        #{"role": "assistant", "content": ex["chinese"]},
    ]
    text = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=False
    )
    return {"text": text}


def tokenize_example(ex, tokenizer, max_length=2048):
    tokens = tokenizer(
        ex["text"],
        truncation=True,
        max_length=max_length,
    )
    return {
        "input_ids": tokens["input_ids"],
        "attention_mask": tokens["attention_mask"],
    }

# apply to dataset
train_ds = train_ds.map(
    format_example, fn_kwargs={"tokenizer": tokenizer},
    remove_columns=train_ds.column_names
).map(
    tokenize_example, fn_kwargs={"tokenizer": tokenizer},
    remove_columns=["text"]
)

val_ds = val_ds.map(
    format_example, fn_kwargs={"tokenizer": tokenizer},
    remove_columns=val_ds.column_names
).map(
    tokenize_example, fn_kwargs={"tokenizer": tokenizer},
    remove_columns=["text"]
)

test_ds = test_ds.map(
    format_example, fn_kwargs={"tokenizer": tokenizer},
    remove_columns=test_ds.column_names
).map(
    tokenize_example, fn_kwargs={"tokenizer": tokenizer},
    remove_columns=["text"]
)


Map: 100%|██████████████████████████████████████████████████████████████████| 473/473 [00:00<00:00, 3307.02 examples/s]


In [5]:
# ====
# LoRa
# ====
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    lora_dropout=0.0,
    bias="none",
    use_gradient_checkpointing=True,
)


Unsloth 2025.11.4 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


In [7]:
# ===============
# training config
# ===============

from trl import SFTTrainer, SFTConfig
import torch, datasets
import os

sft_config = SFTConfig(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=20,
    max_steps=400,
    logging_steps=1,
    output_dir=r"C:\Users\timot\Downloads\nlp training\qwen3-4b-dialogue-lora",
    optim="adamw_8bit",
    num_train_epochs=2,
    learning_rate=3.7e-4,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    completion_only_loss=False,
    eval_strategy="steps",
    eval_steps=20,
)

os.environ["TOKENIZERS_PARALLELISM"] = "false"
datasets.config.DEFAULT_MAX_BATCH_SIZE = 1

In [8]:
# =======
# trainer
# =======
trainer = SFTTrainer(
    model=model,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    args=sft_config,
)


In [9]:
# unsloth config
import torch
torch._dynamo.config.suppress_errors = True
torch._dynamo.config.disable = True

In [10]:
# =====
# train
# =====
trainer.train()

# ===========================
# save final loRa + tokenizer
# ===========================
save_dir = r"C:\Users\timot\Downloads\nlp training\qwen3-4b-dialogue-lora\final"
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)


The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 8,023 | Num Epochs = 1 | Total steps = 400
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 33,030,144 of 4,055,498,240 (0.81% trained)


Step,Training Loss,Validation Loss
20,2.272100,2.309618
40,1.562400,1.730411
60,1.504000,1.686203
80,1.739000,1.657347
100,1.409400,1.641471
120,1.719400,1.639650
140,1.707900,1.613685
160,1.847500,1.589851
180,1.760100,1.597790
200,1.634400,1.572113


Unsloth: Not an error, but Qwen3ForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


Unsloth: Will smartly offload gradients to save VRAM!


('C:\\Users\\timot\\Downloads\\nlp training\\qwen3-4b-dialogue-lora\\final\\tokenizer_config.json',
 'C:\\Users\\timot\\Downloads\\nlp training\\qwen3-4b-dialogue-lora\\final\\special_tokens_map.json',
 'C:\\Users\\timot\\Downloads\\nlp training\\qwen3-4b-dialogue-lora\\final\\chat_template.jinja',
 'C:\\Users\\timot\\Downloads\\nlp training\\qwen3-4b-dialogue-lora\\final\\vocab.json',
 'C:\\Users\\timot\\Downloads\\nlp training\\qwen3-4b-dialogue-lora\\final\\merges.txt',
 'C:\\Users\\timot\\Downloads\\nlp training\\qwen3-4b-dialogue-lora\\final\\added_tokens.json',
 'C:\\Users\\timot\\Downloads\\nlp training\\qwen3-4b-dialogue-lora\\final\\tokenizer.json')

In [11]:
# ====
# eval
# ====
metrics = trainer.evaluate(test_ds)
print(metrics)

{'eval_loss': 1.4759572744369507, 'eval_runtime': 21.234, 'eval_samples_per_second': 22.276, 'eval_steps_per_second': 5.604, 'epoch': 0.3988035892323031}


In [12]:
# ==========
# eval setup
# ==========
!pip install -q evaluate seqeval sentence-transformers
import evaluate
import torch
from tqdm.auto import tqdm

from unsloth import FastLanguageModel
FastLanguageModel.for_inference(model)



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen3ForCausalLM(
      (model): Qwen3Model(
        (embed_tokens): Embedding(151936, 2560, padding_idx=151654)
        (layers): ModuleList(
          (0-1): 2 x Qwen3DecoderLayer(
            (self_attn): Qwen3Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2560, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2560, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(


In [17]:
# =================
# generation helper
# =================

# def generate_english(chinese: str) -> str:
#     prompt = tokenizer.apply_chat_template(
#         [{"role": "user", "content": chinese}],  # ← Chinese input
#         tokenize=False,
#         add_generation_prompt=True
#     )
#     inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
#     with torch.no_grad():
#         out = model.generate(
#             **inputs,
#             max_new_tokens=128,
#             do_sample=False,
#             pad_token_id=tokenizer.eos_token_id
#         )
#     gen = out[0][inputs.input_ids.shape[-1]:]
#     return tokenizer.decode(gen, skip_special_tokens=True).strip()


def generate_english(chinese: str) -> str:
    system_prompt = (
        "You are a faithful Chinese→English translator. "
        "Follow these rules:\n"
        "1. Do not infer gender unless the source explicitly states it.\n"
        "2. Translate faithfully; do not add new insults or escalate rudeness.\n"
        "3. Do not reinterpret sensitive terms—keep original nuance.\n"
        "4. Output only the English translation, no explanations."
    )
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": chinese}
    ]
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=128,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )
    gen = out[0][inputs.input_ids.shape[-1]:]
    return tokenizer.decode(gen, skip_special_tokens=True).strip()


In [14]:
# =============================
# run trained model on test set
# =============================
preds, refs = [], []
for ex in tqdm(raw_test):
    preds.append(generate_english(ex["chinese"]))
    refs.append(ex["english"])

# for en to zh
# preds, refs = [], []
# for ex in tqdm(raw_test):
#     preds.append(generate_chinese(ex["english"]))
#     refs.append(ex["chinese"])

100%|████████████████████████████████████████████████████████████████████████████████| 473/473 [14:54<00:00,  1.89s/it]


In [15]:
# =================================
# f1 and semantic similarity report
# =================================
from collections import Counter
from sentence_transformers import SentenceTransformer, util

# ------- character-level F1 (bag of words similarity) -------
def char_f1(preds, refs):
    true_counter = Counter(''.join(refs))
    pred_counter = Counter(''.join(preds))
    tp = sum((true_counter & pred_counter).values())
    prec = tp / max(sum(pred_counter.values()), 1e-9)
    rec  = tp / max(sum(true_counter.values()), 1e-9)
    return 2 * prec * rec / max(prec + rec, 1e-9)

# ---------- semantic similarity ----------
model_sim = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
emb_pred = model_sim.encode(preds, convert_to_tensor=True, normalize_embeddings=True)
emb_ref  = model_sim.encode(refs,  convert_to_tensor=True, normalize_embeddings=True)
sbert_scores = util.pytorch_cos_sim(emb_pred, emb_ref).diag().cpu().tolist()

print(f"Character-level F1 : {char_f1(preds, refs):.2%}")
print(f"SBERT similarity   : {sum(sbert_scores)/len(sbert_scores):.2%}")

Character-level F1 : 72.31%
SBERT similarity   : 68.97%


In [16]:
zh_test_lines = {
    "sarcasm": [
        "哎呀，又是周一早会，真是太‘惊喜’了。",
        "您可真是个天才——下次记得看说明书哦。",
        "哇，这网速真快，我都快睡着了。",
        "您这‘高效率’真让我大开眼界。",
        "太好了，又加班到十点，人生巅峰啊。",
    ],
    "anger": [
        "立刻滚出我的办公室，马上！",
        "你再碰一下，我发誓打断你的手。",
        "别再让我看到你，滚！",
        "你脑子进水了吗？",
        "立刻给我消失，否则后果自负！",
    ],
    "ambiguous_tense_aspect": [
        "我吃了苹果。",          # ate / have eaten / had eaten?
        "她在写报告。",          # is writing / was writing / will be writing?
        "我写了信。",          # wrote / have written / had written?
        "他们正在吃饭。",      # are eating / were eating / will be eating?
        "他走了。",            # left / has left / had left?
    ],
    "implicit_gender": [
        "医生说我要休息一周。",     # doctor gender unspecified
        "护士每天早上给我端茶。",   # nurse gender unspecified
        "CEO昨天宣布了新政。",      # CEO gender unspecified
        "秘书说会议改到三点。",      # secretary gender unspecified
        "司机在楼下等。",            # driver gender unspecified
        "律师通知我们胜诉了。",      # lawyer gender unspecified
    ],
    "honorific_inflation": [
        "请经理签一下。",         #  “Manager, please sign”
        "告诉助理订个房间。",     #  “Tell the assistant to…” → “Tell Assistant Zhang”
        "请总监过目。",          #  “Director, please look”
        "让秘书联系客户。",      # “Secretary Zhang will contact”
        "已发给副总审批。",      # “Vice-President approved”
    ],
    "chengyu_idiom": [
        "他露出了马脚。",         # “showed the horse’s hoof” → spill the beans
        "那段演讲真是催人泪下。", # “makes people shed tears” → tear-jerker
        "她脑子有点不正常。",     # literal: brain not normal → “has a screw loose”
        "他简直杞人忧天。",       # “like the man of Qi who feared the sky would fall” → needless worry
        "别班门弄斧了。",         # “show off your axe in front of Lu Ban” → teach a fish to swim
        "这事已经木已成舟。",      # “the wood is already a boat” → what’s done is done
    ],
    "cultural_reference": [
        "我们不在堪萨斯了。",     # direct loan of English idiom
        "他在那场辩论中遭遇了滑铁卢。",  # “met his Waterloo”
        "听起来像《黑镜》里的剧情。",   # Black Mirror reference
        "他就像堂吉诃德一样冲向风车。",  # Don Quixote reference
        "这操作堪比《甄嬛传》的宫斗。",  # Empresses in the Palace reference
        "简直是现实版《楚门的世界》。",   # Truman Show reference
    ],
    "harmful_direct_speech": [
        "你这个蠢货，我说过别碰它！",
        "闭嘴，死基佬，没人问你！",
        "废物，连这点事都做不好！",
        "滚开，死胖子，挡住路了！",
        "臭女人，少在这里装清高！",
    ],
    "classifier_mismatch": [
        "一辆车",   # measure word 辆 → risk: “a car” vs “one car”
        "一条狗",   # 条 → “a dog” (not “one strip of dog”)
        "一张桌子",   # 张 → “a table” (not “one sheet of table”)
        "一只猫",     # 只 → “a cat” (not “one hand of cat”)
        "一颗星星",   # 颗 → “a star” (not “one grain of star”)
    ],
    "zero_pronoun": [
        "看见了，很喜欢，就买了。",  # no explicit subject → “[I] saw it, liked it, and bought it.”
        "打开电视，发现播广告，就关了。",
        "收到邮件，读完，直接删除。",
        "路过书店，进去逛了半小时。",
    ],
    "relationship_explicitness": [
        "我哥哥昨天结婚。",  # 哥哥 → older brother
        "她姐姐在银行工作。", # 姐姐 → older sister
        "我堂哥在国外定居。",    # 堂哥 → older male cousin (father’s side)
    ],
}

for category, lines in zh_test_lines.items():
    print(f"--- ZH→EN  {category} ---\n")
    for zh in lines:
        inputs = tokenizer.apply_chat_template(
            [{"role": "user", "content": zh}],
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt"
        ).to("cuda")
        out = model.generate(input_ids=inputs, max_new_tokens=128, temperature=0.7, do_sample=True)
        en = tokenizer.decode(out[0][inputs.shape[-1]:], skip_special_tokens=True)
        print("ZH:", zh)
        print("EN:", en)
        print()

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


--- ZH→EN  sarcasm ---

ZH: 哎呀，又是周一早会，真是太‘惊喜’了。
EN: <think>

</think>

Oh, it's a Monday morning meeting, too.

ZH: 您可真是个天才——下次记得看说明书哦。
EN: <think>

</think>

You’re a genius, though.  Please, next time, please, please, please, please, please, please, please, please, please, please, please, please, please, please, please, please, please, please, please, please, please, please, please, please, please, please, please, please, please, please, please, please, please, please, please, please, please, please, please, please, please, please, please, please, please, please, please, please, please, please, please, please, please, please, please, please

ZH: 哇，这网速真快，我都快睡着了。
EN: <think>

</think>

Wow, that’s a fast connection. I’m so sleepy.

ZH: 您这‘高效率’真让我大开眼界。
EN: <think>

</think>

and I’m impressed with the efficiency.

ZH: 太好了，又加班到十点，人生巅峰啊。
EN: <think>

</think>

That’s good, working overtime till ten, man.

--- ZH→EN  anger ---

ZH: 立刻滚出我的办公室，马上！
EN: <think>

</think>

Get out of my office 

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(save_dir)

In [ ]:
# ===========================
# save hf format LoRa adapter
# ===========================
model.save_pretrained("my-lora-adapter")   # HF format
tokenizer.save_pretrained("my-lora-adapter")

# load:
"""
model, tokenizer = FastLanguageModel.from_pretrained(
    "my-lora-adapter",
    load_in_4bit=True,
)
"""

'\nmodel, tokenizer = FastLanguageModel.from_pretrained(\n    "my-lora-adapter",\n    load_in_4bit=True,\n)\n'